In [33]:
# Importing the libraries

from typing import Optional, Tuple

import torch
import torch.nn as nn
from torch.nn import functional as F

In [29]:
torch.manual_seed(1337)

In [4]:
# Loading and printing the first 1000 characters of the text

with open("../data/tinyshakespeare.txt", "r") as f:
    text = f.read()

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [7]:
# Extracting the vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [16]:
# Creating a dictionary to map the characters to integers (tokenizers)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda x: [stoi[ch] for ch in x]
decode = lambda x: "".join([itos[ch] for ch in x])

print(encode("Hello, students!"))
print(decode(encode("Hello, students!")))

[20, 43, 50, 50, 53, 6, 1, 57, 58, 59, 42, 43, 52, 58, 57, 2]
Hello, students!


In [20]:
# Encoding part of the text

data = torch.tensor(encode(text[:1000]), dtype=torch.int64)
data

/var/folders/3t/wmv3mchd6w99pw9syrlwn5kh0000gq/T/ipykernel_12387/900528017.py:3: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  data = torch.tensor(encode(text[:1000]), dtype=torch.int64)


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
        47, 59, 57,  1, 47, 57,  1, 41, 

In [21]:
# Encoding the whole text
data = torch.tensor(encode(text), dtype=torch.int64)

In [22]:
# Splitting the data into training and validation sets
n = int(len(data) * 0.9)
train_data = data[:n]
val_data = data[n:]

In [27]:
# Checking input to the model + 1
block_size = 8
train_data[: block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [30]:
# Printing the input and target sequences

x = train_data[:block_size]
y = train_data[1 : block_size + 1]

for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"when input is {context}, the target is {target}")

when input is tensor([18]), the target is 47
when input is tensor([18, 47]), the target is 56
when input is tensor([18, 47, 56]), the target is 57
when input is tensor([18, 47, 56, 57]), the target is 58
when input is tensor([18, 47, 56, 57, 58]), the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]), the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]), the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]), the target is 58


In [32]:
# Contructing the batches

batch_size = 4
block_size = 8


def get_batch(split: str) -> Tuple[torch.Tensor, torch.Tensor]:
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)

print("---------")

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()}, the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[54, 43, 39, 49,  8,  0,  0, 18],
        [46, 43, 63,  1, 58, 46, 47, 52],
        [ 0,  0, 18, 47, 56, 57, 58,  1],
        [45, 53, 53, 42,  1, 41, 47, 58]])
targets:
torch.Size([4, 8])
tensor([[43, 39, 49,  8,  0,  0, 18, 47],
        [43, 63,  1, 58, 46, 47, 52, 49],
        [ 0, 18, 47, 56, 57, 58,  1, 15],
        [53, 53, 42,  1, 41, 47, 58, 47]])
---------
when input is [54], the target is 43
when input is [54, 43], the target is 39
when input is [54, 43, 39], the target is 49
when input is [54, 43, 39, 49], the target is 8
when input is [54, 43, 39, 49, 8], the target is 0
when input is [54, 43, 39, 49, 8, 0], the target is 0
when input is [54, 43, 39, 49, 8, 0, 0], the target is 18
when input is [54, 43, 39, 49, 8, 0, 0, 18], the target is 47
when input is [46], the target is 43
when input is [46, 43], the target is 63
when input is [46, 43, 63], the target is 1
when input is [46, 43, 63, 1], the target is 58
when input is [46, 43, 63, 1, 5

In [56]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size: int) -> None:
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(
        self, idx: torch.Tensor, targets: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        logits = self.token_embedding_table(
            idx
        )  # (batch_size, block_size, vocab_size) -> (B,T,C)

        # If targets is None, we are in inference mode
        if targets is None:
            return logits, None

        B, T, C = logits.shape
        logits = logits.view(B * T, C)  # (batch_size * block_size, vocab_size)
        targets = targets.view(-1)  # (batch_size * block_size)

        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx: torch.Tensor, max_new_tokens: int) -> torch.Tensor:
        """
        Generates new tokens given a sequence of tokens.

        Args:
            idx: (batch_size, block_size) tensor containing the tokens
            max_new_tokens: maximum number of tokens to generate

        Returns:
            (batch_size, block_size + max_new_tokens) tensor containing the original tokens and the generated tokens
        """
        for _ in range(max_new_tokens):
            logits, _ = self(idx)

            logits = logits[:, -1, :]  # (batch_size, vocab_size)
            probs = F.softmax(logits, dim=-1)  # (batch_size, vocab_size)

            idx_next = torch.multinomial(probs, 1)
            idx = torch.cat([idx, idx_next], dim=1)

        return idx

In [65]:
# Checking out the model without training

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.int64)
print(decode(m.generate(idx, max_new_tokens=10)[0].tolist()))

torch.Size([256, 65])
tensor(4.5100, grad_fn=<NllLossBackward0>)

llfIJ
uaAb


In [66]:
# Configuring the optimizer

optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [67]:
# Defining the training loop
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.023599624633789


In [68]:
idx = torch.zeros((1, 1), dtype=torch.int64)
print(decode(m.generate(idx, max_new_tokens=500)[0].tolist()))


Fit wecir uis d Cis, led lve t pars t wn ustheshe ow'l, l penXzear
Noulafayitird.
bellved t ars tis antild plus ansondave itho
Fitieaunghe'lve ce paice Mansuthe t.

Fiecushabun:
Fizes: urestilvex



Wes Mactoussules thise wen rstopey sorthit an: atine
Wer, ir:

S; d ito metht iusurf t u k wen't ictqffourthey; hacceve ghirs bjesus't.
Le:
so am, ren Cincous m, uiknmHx3Poonne rcthum, reranerizeakit ons Citit, Cizer as ey, ary!



Wery, ut oris r:
Wed s kin:
Bemichen:
wn:
Ald orspl:
ispa or thaw'the
